# Install GPU related stuff

In [1]:
!pip install cudf cuml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 MB 15.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.12.2 requires dask<=2021.11.2,>=2021.11.1, but you have dask 2022.2.0 which is incompatible.


# Import necessary libraries

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cudf
import cuml
from cuml.model_selection import train_test_split
from cuml.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tumai-x-aimunich-x-kkcompany/train_target_events.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/sample.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/light_train_target_labels.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/light_train_source_labels.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/test_source_events.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/train_source_events.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/light_test_source_labels.csv


# Prepare Data

In [3]:
time_slot_data_X = cudf.read_csv('/kaggle/input/tumai-x-aimunich-x-kkcompany/light_train_source_labels.csv')
time_slot_data_y = cudf.read_csv('/kaggle/input/tumai-x-aimunich-x-kkcompany/light_train_target_labels.csv')

In [4]:
time_slot_cols_X = list(time_slot_data_X.columns)
time_slot_cols_X.remove('user_id')

time_slot_cols_y = list(time_slot_data_y.columns)
time_slot_cols_y.remove('user_id')


X = time_slot_data_X[time_slot_cols_X]
y = time_slot_data_y[time_slot_cols_y]

# split into training and testing sets
X_train, X_test, y_train_cudf, y_test_cudf = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = cudf.DataFrame(X_train)
X_test = cudf.DataFrame(X_test)

y_train = cudf.DataFrame(y_train_cudf)
y_test = cudf.DataFrame(y_test_cudf)

# Train XG-Boost Classifier

In [5]:
# create XGBoost classifier
clf = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0)

# fit the classifier to the training data
clf.fit(X_train, y_train)

# make predictions on the testing data
y_pred = clf.predict(X_test)

# calculate accuracy of the predictions
accuracy = cuml.accuracy_score(y_test.values.ravel(), y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.772488534450531


# Create Submission

In [6]:
X_submission = cudf.read_csv('/kaggle/input/tumai-x-aimunich-x-kkcompany/light_test_source_labels.csv')
user_ids = X_submission['user_id']
X_sub = X_submission[time_slot_cols_X]
y_sub = clf.predict(X_sub)
submission = pd.DataFrame(y_sub, columns=time_slot_cols_y)
submission['user_id'] = user_ids.to_numpy()
submission = submission[['user_id']+time_slot_cols_y]

In [7]:
submission.to_csv('submission.csv', index=False)